DE donde son las localizaciones


Primero todo manual luego nos alocamos con el GAN robado

Teniendo en cuenta que hay datos atipicos presentes y que no nos podemos deshacer de ellos. Usaremos escaladores robustos o transformadores [Sklearn-Doc](https://qu4nt.github.io/sklearn-doc-es/modules/preprocessing.html) 

In [32]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [33]:
df = pd.read_csv('./data/credit_card_transactions.csv', encoding='latin1') 
print(df.shape)
df.head()


(10000, 11)


,TransactionID,Time,Amount,CardholderName,MerchantName,MCC,Location,Device,IPAddress,TransactionSpeed,Fraud
0,1,2025-03-09 06:23:57,83.54,Sarah Murray,Oconnor-Hughes,5411,Longtown,Android,102.247.49.50,38.17,0
1,2,2025-02-14 05:17:36,805.88,Alexander Brooks,Reed PLC,5261,Woodsland,POS,153.157.50.7,49.84,0
2,3,2024-05-03 15:28:52,450.49,Harold Perez,Berry LLC,5261,Jessicatown,iOS,151.174.154.123,48.79,0
3,4,2024-05-03 09:01:14,782.39,Rebecca Davis,Berg Inc,5968,North Krystalburgh,Android,130.70.79.149,8.71,0
4,5,2025-04-24 17:24:39,713.02,Lauren Benson DDS,King-Gaines,5968,East Danielletown,Android,195.136.16.190,17.16,0


In [34]:
df['Time'] = pd.to_datetime(df['Time'])
col_exclude = ['Time', 'CardholderName', 'MerchantName', 'Location', 'IPAddress', 'TransactionID']
df.drop(columns=col_exclude, inplace=True)
df

df

,Amount,MCC,Device,TransactionSpeed,Fraud
0,83.54,5411,Android,38.17,0
1,805.88,5261,POS,49.84,0
2,450.49,5261,iOS,48.79,0
3,782.39,5968,Android,8.71,0
4,713.02,5968,Android,17.16,0
...,...,...,...,...,...
9995,659.56,5261,iOS,1.25,0
9996,290.46,5968,Android,11.04,0
9997,337.55,5812,iOS,27.00,0
9998,322.15,7299,iOS,3.26,0


In [35]:


var_one_hot = ['Device', 'MCC']

In [36]:
df.dtypes

Amount              float64
MCC                   int64
Device               object
TransactionSpeed    float64
Fraud                 int64
dtype: object

## Seleccion de caracteristicas relevantes

Se propone extraer caracterisitcas, de la variable Time, corresponiente a un a: `Month`,`Day`,`WeekDay`, `Hour`.   



Como se dan las incidencias de cada una de las caracteristicas que quiero extraer. 

In [37]:
# seleccionamos las caracteristicas relevantes para predicir la variable objetivo

X = df.iloc[:,:-1]
X.head()

,Amount,MCC,Device,TransactionSpeed
0,83.54,5411,Android,38.17
1,805.88,5261,POS,49.84
2,450.49,5261,iOS,48.79
3,782.39,5968,Android,8.71
4,713.02,5968,Android,17.16


In [38]:

Y = df.iloc[:,-1]
Y.describe()

count    10000.000000
mean         0.019200
std          0.137234
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: Fraud, dtype: float64

## Codificación de variables categóricas (one-hot, label encoding)


In [39]:
# one-hot encoding
X_one_hot = pd.get_dummies(X, columns=var_one_hot, drop_first=True)
X_one_hot


,Amount,TransactionSpeed,Device_POS,Device_Web,Device_iOS,MCC_5411,MCC_5812,MCC_5942,MCC_5968,MCC_7299
0,83.54,38.17,False,False,False,True,False,False,False,False
1,805.88,49.84,True,False,False,False,False,False,False,False
2,450.49,48.79,False,False,True,False,False,False,False,False
3,782.39,8.71,False,False,False,False,False,False,True,False
4,713.02,17.16,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...
9995,659.56,1.25,False,False,True,False,False,False,False,False
9996,290.46,11.04,False,False,False,False,False,False,True,False
9997,337.55,27.00,False,False,True,False,True,False,False,False
9998,322.15,3.26,False,False,True,False,False,False,False,True


Determinar si hay un orden jerarquico en MCC y aplicar label encoding de ser el caso

One-hot 

## Normalización de variables numéricas (MinMaxScaler, StandardScaler)


In [40]:
from sklearn.preprocessing import StandardScaler

#Normalizacion 

scaler = StandardScaler()
normalized_X = scaler.fit_transform(X_one_hot)



## División del dataset: entrenamiento, validación y prueba (70-15-15)

In [41]:
#- [ ] División del dataset: entrenamiento, validación y prueba (70-15-15)
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(normalized_X, Y, test_size=0.15, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=(15/85), random_state=42)


n = X_train.shape[0] + X_val.shape[0] + X_test.shape[0]
print(f"X_train: {X_train.shape[0]/n:2}%, y_train: {y_train.shape[0]}")
print(f"X_val: {X_val.shape[0]/n:2}%, y_val: {y_val.shape[0]}")
print(f"X_test: {X_test.shape[0]/n:2}%, y_test: {y_test.shape[0]}")

X_train: 0.7%, y_train: 7000
X_val: 0.15%, y_val: 1500
X_test: 0.15%, y_test: 1500


In [42]:
# guardamos los datos
X_train = pd.DataFrame(X_train, columns=X_one_hot.columns)
X_val = pd.DataFrame(X_val, columns=X_one_hot.columns)
X_test = pd.DataFrame(X_test, columns=X_one_hot.columns)

Y_train = pd.Series(y_train, name='is_fraud')
Y_val = pd.Series(y_val, name='is_fraud')
Y_test = pd.Series(y_test, name='is_fraud')

import os

data_path = './data/processed/'

if not os.path.exists(f'{data_path}'):
    os.makedirs(f'{data_path}')

X_train.to_csv(f'{data_path}/X_train.csv', index=False)
X_val.to_csv(f'{data_path}/X_val.csv', index=False)
X_test.to_csv(f'{data_path}/X_test.csv', index=False)
Y_train.to_csv(f'{data_path}/Y_train.csv', index=False)
Y_val.to_csv(f'{data_path}/Y_val.csv', index=False)
Y_test.to_csv(f'{data_path}/Y_test.csv', index=False)

## Manejo del desbalance de clases (sobremuestreo, submuestreo, pesos de clase)

posemos un submuestro de transacciones fraudulentas

In [43]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE al conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, Y_train)

print(f"Antes del sobremuestreo: {Y_train.value_counts()}")
print(f"Después del sobremuestreo: {y_train_resampled.value_counts()}")

save = True

if (save):
    # guardamos los datos
    X_train = pd.DataFrame(X_train, columns=X_one_hot.columns)
    X_val = pd.DataFrame(X_val, columns=X_one_hot.columns)

    Y_train = pd.Series(y_train, name='is_fraud')
    Y_val = pd.Series(y_val, name='is_fraud')

    import os

    data_path = './data/processed/'

    if not os.path.exists(f'{data_path}'):
        os.makedirs(f'{data_path}')

    X_train.to_csv(f'{data_path}/X_train.csv', index=False)
    X_val.to_csv(f'{data_path}/X_val.csv', index=False)
    X_test.to_csv(f'{data_path}/X_test.csv', index=False)
    Y_train.to_csv(f'{data_path}/Y_train.csv', index=False)
    Y_val.to_csv(f'{data_path}/Y_val.csv', index=False)
    Y_test.to_csv(f'{data_path}/Y_test.csv', index=False)

Antes del sobremuestreo: is_fraud
0    6869
1     131
Name: count, dtype: int64
Después del sobremuestreo: is_fraud
0    6869
1    6869
Name: count, dtype: int64


In [45]:
from imblearn.under_sampling import RandomUnderSampler

# Aplicar submuestreo al conjunto de entrenamiento
rus = RandomUnderSampler(random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, Y_train)

print(f"Antes del submuestreo: {Y_train.value_counts()}")
print(f"Después del submuestreo: {y_train_resampled.value_counts()}")



Antes del submuestreo: is_fraud
0    6869
1     131
Name: count, dtype: int64
Después del submuestreo: is_fraud
0    131
1    131
Name: count, dtype: int64


In [46]:

from sklearn.ensemble import RandomForestClassifier

# Entrenar un modelo con ajuste de pesos de clase
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, Y_train)

# Evaluar el modelo
print(f"Score del modelo: {model.score(X_val, Y_val)}")


Score del modelo: 1.0


In [47]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_val)
print(confusion_matrix(Y_val, y_pred))
print(classification_report(Y_val, y_pred))

[[1466    0]
 [   0   34]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1466
           1       1.00      1.00      1.00        34

    accuracy                           1.00      1500
   macro avg       1.00      1.00      1.00      1500
weighted avg       1.00      1.00      1.00      1500



In [48]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, normalized_X, Y, cv=5, scoring='f1')
print(f"F1-scores en validación cruzada: {scores}")
print(f"F1-score promedio: {scores.mean()}")

F1-scores en validación cruzada: [1. 1. 1. 1. 1.]
F1-score promedio: 1.0
